In [1]:
# merge all training and testing file into one file respectively
#!for split in train test; do for sentiment in pos neg; do for file in aclImdb/$split/$sentiment/*; do cat $file >> aclImdb/movie_data/full_${split}_$sentiment.txt; echo >> aclImdb/movie_data/full_${split}_$sentiment.txt; done; done; done;

In [1]:
!head -1 ./aclImdb/movie_data/full_train_pos.txt

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [3]:
#build train_data_tokens.p, all_train_tokens.p, val_data_tokens.p, test_data_tokens.p
#!python load_data.py   

In [2]:
import numpy as np
import pickle as pkl
from module import *
import prep
import load_data
%load_ext autoreload
%autoreload 2


prepath_data = './aclImdb/movie_data/'


Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


In [49]:
import numpy as np
from module import *
import prep

prepath_data = './aclImdb/movie_data/'
tp_filename = 'full_train_pos.txt'   #train_pos_filename
tn_filename = 'full_train_neg.txt'   #train_neg_filename
tep_filename = 'full_test_pos.txt'    #test_pos_filename
ten_filename = 'full_test_neg.txt'    #test_neg_filename



movie_train_data, movie_train_target = prep.read_data(prepath_data + tp_filename, prepath_data + tn_filename)
movie_test_data, movie_test_target = prep.read_data(prepath_data + tep_filename, prepath_data + ten_filename)

train_split = 20000
train_data = movie_train_data[:train_split]
train_targets = movie_train_target[:train_split]

val_data = movie_train_data[train_split:]
val_targets = movie_train_target[train_split:]

test_data = movie_test_data
test_targets = movie_test_target

print ("Train dataset size is {}".format(len(train_data)))
print ("Val dataset size is {}".format(len(val_data)))
print ("Test dataset size is {}".format(len(test_data)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


In [ ]:
#n-gram features for texts
def f_ngram(train, test, mode='tfidf', binary=1, ngram=(1,1), min_c=1):
    '''exact n-gram feacturs
    return: feature array (fgram); feature vocabulary (vocab)
    input: raw train and test data; ngram = (n,n) denote n_gram and ngram=(1,2) denote
    1_gram and 2_gram; tokens with count below min_c are cut off. 
    '''
    if mode == 'tfidf':
        if binary==1:
            gram = text.TfidfVectorizer(ngram_range=ngram, binary=True, min_df=min_c)
        else:
            gram = text.TfidfVectorizer(ngram_range=ngram, min_df=min_c)
        
    else: #mode=count
        if binary==1:
            gram = text.CountVectorizer(ngram_range=ngram, binary=True, min_df=min_c)
        else:
            gram = text.CountVectorizer(ngram_range=ngram, min_df=min_c)
    train = dataFilter_train(train)
    gram = gram.fit(train)
    vocab = gram.get_feature_names()
    fgram_train = gram.transform(train).toarray()
    test = dataFilter(test, vocab)
    fgram_test = gram.transform(test).toarray()
    return (fgram_train, fgram_test, vocab)


In [50]:
#val set tokens
print ("Tokenizing val data")
val_data_tokens, _ = prep.tokenize_dataset(val_data)
pkl.dump(val_data_tokens, open(prepath_data + "val_data_tokens.p", "wb"))

#test set tokens
print ("Tokenizing test data")
test_data_tokens, _ = prep.tokenize_dataset(test_data)
pkl.dump(test_data_tokens, open(prepath_data + "test_data_tokens.p", "wb"))

#train set tokens
print ("Tokenizing train data")
train_data_tokens, all_train_tokens = prep.tokenize_dataset(train_data)
pkl.dump(train_data_tokens, open(prepath_data + "train_data_tokens.p", "wb"))
pkl.dump(all_train_tokens, open(prepath_data + "all_train_tokens.p", "wb"))

Tokenizing val data
Tokenizing test data
Tokenizing train data


In [9]:
# load preprocessed train, val and test datasets
train_data_tokens = pkl.load(open(prepath_data + "train_data_tokens.p", "rb"))
all_train_tokens = pkl.load(open(prepath_data + "all_train_tokens.p", "rb"))

val_data_tokens = pkl.load(open(prepath_data + "val_data_tokens.p", "rb"))
test_data_tokens = pkl.load(open(prepath_data + "test_data_tokens.p", "rb"))

# double checking
print ("Train dataset size is {}".format(len(train_data_tokens)))
print ("Val dataset size is {}".format(len(val_data_tokens)))
print ("Test dataset size is {}".format(len(test_data_tokens)))

print ("Total number of tokens in train dataset is {}".format(len(all_train_tokens)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000
Total number of tokens in train dataset is 4742387


Now, we are going to create the vocabulary of most common 10,000 tokens in the training set.

In [51]:
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)

In [53]:
# Lets check the dictionary by loading random token from it
import random
random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 7820 ; token life as
Token life as; token id 7820


In [54]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_data_indices = token2index_dataset(train_data_tokens)
val_data_indices = token2index_dataset(val_data_tokens)
test_data_indices = token2index_dataset(test_data_tokens)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(test_data_indices)))

Train dataset size is 20000
Val dataset size is 5000
Test dataset size is 25000


Now we are going to create PyTorch DataLoader 

In [55]:
from newsgroupdataset import NewsGroupDataset

BATCH_SIZE = 32
train_dataset = NewsGroupDataset(train_data_indices, train_targets)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset = NewsGroupDataset(val_data_indices, val_targets)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

test_dataset = NewsGroupDataset(test_data_indices, test_targets)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=False)

#for i, (data, lengths, labels) in enumerate(train_loader):
#    print (data)
#    print (labels)
#    break

Here we will define Bag-of-Words model in PyTorch

In [56]:
# First import torch related libraries
from bagofwords import BagOfWords

emb_dim = 100
model = BagOfWords(len(id2token), emb_dim)

In [58]:
import torch.nn.functional as F

learning_rate = 0.01
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data, lengths, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))


Epoch: [1/10], Step: [101/625], Validation Acc: 60.58
Epoch: [1/10], Step: [201/625], Validation Acc: 76.62
Epoch: [1/10], Step: [301/625], Validation Acc: 75.88
Epoch: [1/10], Step: [401/625], Validation Acc: 77.38
Epoch: [1/10], Step: [501/625], Validation Acc: 68.32
Epoch: [1/10], Step: [601/625], Validation Acc: 70.2
Epoch: [2/10], Step: [101/625], Validation Acc: 80.26
Epoch: [2/10], Step: [201/625], Validation Acc: 90.12
Epoch: [2/10], Step: [301/625], Validation Acc: 76.1
Epoch: [2/10], Step: [401/625], Validation Acc: 74.94
Epoch: [2/10], Step: [501/625], Validation Acc: 80.92
Epoch: [2/10], Step: [601/625], Validation Acc: 83.9
Epoch: [3/10], Step: [101/625], Validation Acc: 71.02
Epoch: [3/10], Step: [201/625], Validation Acc: 68.08
Epoch: [3/10], Step: [301/625], Validation Acc: 77.64
Epoch: [3/10], Step: [401/625], Validation Acc: 72.7
Epoch: [3/10], Step: [501/625], Validation Acc: 77.5
Epoch: [3/10], Step: [601/625], Validation Acc: 87.06
Epoch: [4/10], Step: [101/625], V

In [60]:
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(test_model(val_loader, model)))
print ("Test Acc {}".format(test_model(test_loader, model)))

After training for 10 epochs
Val Acc 77.44
Test Acc 80.328


## Exercise 1
### Try training the model with larger embedding size and for larger number of epochs
### Also plot the training curves of the model

## Exercise 2:
### Try downloading IMDB Large Movie Review Dataset that is used for Assignment 1 http://ai.stanford.edu/~amaas/data/sentiment/
### and tokenize it

## Exercise 3:
### If you have time, after tokenizing the dataset try training Bag-of-Words model on it and report your initial results
### on validation set.